In [73]:
import os
import copy
import shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForMaskedLM, AutoModelForSeq2SeqLM 
import shap
import scipy as sp
import nlp
import torch
from shap.utils import safe_isinstance

### CausalLM

#### GPT2

In [52]:
model = AutoModelForCausalLM.from_pretrained('distilgpt2')
tokenizer = AutoTokenizer.from_pretrained("distilgpt2")

In [59]:
model.config.is_decoder=True

In [79]:
model.config.decoder

AttributeError: 'GPT2Config' object has no attribute 'decoder'

In [38]:
text = "Who was Jim Henson ? Jim Henson was a"
indexed_tokens = tokenizer.encode(text)

# Convert indexed tokens in a PyTorch tensor
tokens_tensor = torch.tensor([indexed_tokens])

In [83]:
[indexed_tokens]

[[8241, 373, 5395, 367, 19069, 5633, 5395, 367, 19069, 373, 257]]

In [40]:
tokens_tensor

tensor([[ 8241,   373,  5395,   367, 19069,  5633,  5395,   367, 19069,   373,
           257]])

In [41]:
model.eval()
outputs = model(tokens_tensor)

In [42]:
outputs[0].shape

torch.Size([1, 11, 50257])

In [44]:
predictions = outputs[0]
predicted_index = torch.argmax(predictions[0, -1, :]).item()
predicted_text = tokenizer.decode(indexed_tokens + [predicted_index])
predicted_text

'Who was Jim Henson? Jim Henson was a great'

In [46]:
model.config.is_decoder

False

In [80]:
model = AutoModelForSeq2SeqLM.from_pretrained('sshleifer/distilbart-xsum-12-6')
tokenizer = AutoTokenizer.from_pretrained("sshleifer/distilbart-xsum-12-6")

In [82]:
model.config.decoder_start_token_id

2

In [67]:
torch.ones((1,1))*43

tensor([[43.]])

In [69]:
model.__class__

transformers.modeling_gpt2.GPT2LMHeadModel

NameError: name 'transformers' is not defined

In [74]:
def safe_isinstance(obj, class_path_str):
    """
    Acts as a safe version of isinstance without having to explicitly
    import packages which may not exist in the users environment.

    Checks if obj is an instance of type specified by class_path_str.

    Parameters
    ----------
    obj: Any
        Some object you want to test against
    class_path_str: str or list
        A string or list of strings specifying full class paths
        Example: `sklearn.ensemble.RandomForestRegressor`

    Returns
    --------
    bool: True if isinstance is true and the package exists, False otherwise
    """
    if isinstance(class_path_str, str):
        class_path_strs = [class_path_str]
    elif isinstance(class_path_str, list) or isinstance(class_path_str, tuple):
        class_path_strs = class_path_str
    else:
        class_path_strs = ['']
    
    # try each module path in order
    for class_path_str in class_path_strs:
        if "." not in class_path_str:
            raise ValueError("class_path_str must be a string or list of strings specifying a full \
                module path to a class. Eg, 'sklearn.ensemble.RandomForestRegressor'")

        # Splits on last occurence of "."
        module_name, class_name = class_path_str.rsplit(".", 1)

        # here we don't check further if the model is not imported, since we shouldn't have
        # an object of that types passed to us if the model the type is from has never been
        # imported. (and we don't want to import lots of new modules for no reason)
        if module_name not in sys.modules:
            continue

        module = sys.modules[module_name]
        
        #Get class
        _class = getattr(module, class_name, None)
        
        if _class is None:
            continue
        
        if isinstance(obj, _class):
            return True

    return False

In [77]:
import sys
safe_isinstance(model,"transformers.PreTrainedModel")

False